
# SNARK : Modélisation Simple (R1CS vers QAP)
Dans ce notebook, nous explorons le processus de conversion d'une équation polynomiale en un système de contraintes R1CS (Rank 1 Constraint Systems) puis en un QAP (Quadratic Arithmetic Programs). 

L'exemple donné est celui où Bob souhaite prouver qu'il connaît un secret \( x \) tel que \( x^3+x+5 = 35 \). Nous allons décomposer cette équation, la modéliser sous forme de circuit arithmétique, puis la convertir en un système R1CS.


In [109]:
import zktool as zk
import numpy as np


## Entrée de l'équation
Veuillez entrer une équation polynomiale de type \( x^3 + x + 5 \). Nous allons ensuite la décomposer et la modéliser sous forme de circuit arithmétique.


In [110]:
eq_input=input("Enter the equation: de type x^3 + x + 5") #METTRE DES COEFFICIENTS >=0, aucune negatif pour l'instant


## Décomposition de l'équation et Modélisation des Circuits Arithmétiques
Une fois que nous avons l'équation, nous pouvons la décomposer en termes individuels et la représenter sous forme de circuit arithmétique. Le résultat ci-dessous montre cette représentation.


In [111]:
equations = zk.decompose_polynomial(eq_input)
unique_words = zk.get_unique_words(equations)

print("Modelisation des circuits arithmetiques :")
for eq in equations:
    print(eq)


print("\nEnsemble des varibles =",unique_words)


Modelisation des circuits arithmetiques :
power_1 = -3 * x
equation_2 =  + power_1
equation_3 = equation_2 + 6

Ensemble des varibles = {'equation_2', 'x', '1', 'power_1', 'equation_3'}



## Conversion du Circuit en R1CS
Le système R1CS (Rank 1 Constraint Systems) est une façon de représenter un ensemble de contraintes sous forme de matrices. Pour chaque équation du circuit, nous allons créer trois vecteurs associés (un pour chaque matrice A, B, C) qui représentent cette équation dans le système R1CS.


In [112]:
#création des 3 matrice vides A B C
A= []
B= []
C= []

for eq in equations:
    print(eq)
print(" ")

ref_array=list(unique_words)
print(ref_array)
print(' ')

for eq in equations:
    eq_split=eq.split()
    #print(eq_split)

    if eq_split[3]=='*':
        vecta=zk.get_position_vector(list(unique_words), eq_split[2])
        A.append(vecta)
        vectb=zk.get_position_vector(list(unique_words), eq_split[4])
        B.append(vectb)
        vectc=zk.get_position_vector(list(unique_words), eq_split[0])
        C.append(vectc)

    if eq_split[3]=='+':
        vecta=zk.get_position_vector(list(unique_words), [eq_split[2],eq_split[4]])
        A.append(vecta)
        vectb=zk.get_position_vector(list(unique_words), "1")
        B.append(vectb)
        vectc=zk.get_position_vector(list(unique_words), eq_split[0])
        C.append(vectc)
        
print("A=")
for ligne in A:
    print(ligne)
print(" ")

print("B=")
for ligne in B:
    print(ligne)
print(" ")

print("C=")
for ligne in C:
    print(ligne)
print(" ")

power_1 = -3 * x
equation_2 =  + power_1
equation_3 = equation_2 + 6
 
['equation_2', 'x', '1', 'power_1', 'equation_3']
 
A=
[0, 0, -3, 0, 0]
[1, 0, 6, 0, 0]
 
B=
[0, 1, 0, 0, 0]
[0, 0, 1, 0, 0]
 
C=
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]
 


## Conversion R1CS en QAP

In [113]:
#Ici on cherche les coefficients des polynomes de chaque matrice

A=np.array(A)
B=np.array(B)
C=np.array(C)

# Convert matrices to their transpose for calculations
At, Bt, Ct = A.transpose(), B.transpose(), C.transpose()

# Calculate polynomial coefficient matrices
PolyA = zk.interpolate_and_get_coeff_matrix(At)
PolyB = zk.interpolate_and_get_coeff_matrix(Bt)
PolyC = zk.interpolate_and_get_coeff_matrix(Ct)

#print exemple
print("[ -0.66666667   5.         -11.33333333   8.        ]")
print("vaut ce polynome -0.6667x^3 + 5.x^2 - 11.33333333*x^1 + 8\n")


# Print the results
print("PolyA:\n", PolyA)
print("\nPolyB:\n", PolyB)
print("\nPolyC:\n", PolyC)


[ -0.66666667   5.         -11.33333333   8.        ]
vaut ce polynome -0.6667x^3 + 5.x^2 - 11.33333333*x^1 + 8

PolyA:
 [[  1.  -1.]
 [  0.   0.]
 [  9. -12.]
 [  0.   0.]
 [  0.   0.]]

PolyB:
 [[ 0.  0.]
 [-1.  2.]
 [ 1. -1.]
 [ 0.  0.]
 [ 0.  0.]]

PolyC:
 [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [-1.  2.]
 [ 1. -1.]]


## Calcul de la preuve

In [114]:
values = {"x": 3}

S = zk.compute_solution_vector(unique_words, equations, values)
print("\nSolution Vector:", S)

# S=[3, 1, 9, 34, 27, 30] #A décommenter pour tester la vérification de la solution fausse


power_1 remains unresolved.


IndexError: list index out of range

-Imaginions si le prouver ne connait pas la solution, il va donner un x qui ne correspond pas à out. Ici x=3 donc la fonction zk.compute_solution_vector va forcement calculer la bonne sortie 35 => Solution Vector: [3, 1, 9, 35, 27, 30].

-Si on veut tester un prouver qui ne connait pas le solution on peux changer manuellement S = [3, 1, 9, **34**, 27, 30]. Maintenant il suffit d'aller voir la fin du programme pour voir si le verifier approuve ou non la preuve.

In [ ]:
#Ici on chercher trouver le polynome T qui repond à tous les polynomes de la matrice A;B;C
result_AS = zk.polynomial_dot_product(PolyA, S)
result_BS = zk.polynomial_dot_product(PolyB, S)
result_CS = zk.polynomial_dot_product(PolyC, S)

print("A.S =", result_AS)
print("B.S =", result_BS)
print("C.S =", result_CS)

# Calcul de la convolution
convolution_product = np.convolve(result_AS, result_BS)

# Calcul de T
T = convolution_product - np.concatenate((result_CS, np.zeros(len(convolution_product) - len(result_CS))))

print("T =", T)


A.S = [-8. 10.]
B.S = [ 5. -2.]
C.S = [0. 6.]
T = [-40.  60. -20.]


In [ ]:
# Z est le diviseur de T pour verifier s'il y a un reste ou non
Z = zk.polynomial_coefficients(len(equations))
print("Z =", Z)

Z = [ 2 -3  1]


In [ ]:
quotient,remainder=zk.polynomial_division(T, Z)

# Imprimer le quotient
print(f"h = T / Z = {quotient}")
print(f"R = T % Z = {remainder}")

# Vérifiez si le reste est nul
if not zk.is_remainder_close_to_zero(remainder, 1e-5):
    print("La preuve est refutée !")
else:
    print("La preuve est bonne")

h = T / Z = [-20.]
R = T % Z = [0.]
La preuve est bonne
